## Scrapeo de página de actividades para Roma y París

In [5]:
from bs4 import BeautifulSoup
import requests
import numpy as np
from tqdm import tqdm
from time import sleep
import math

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd


# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [2]:
driver = webdriver.Chrome()
lista_urls = ["https://www.guias.travel/tour/?s=roma", "https://www.guias.travel/tour/?s=paris"]
ciudades=["roma", "paris"]

j=0
for url in lista_urls:
    driver.get(url)
    driver.maximize_window()
    sleep(1)

    # Veamos cuantas actividades hay
    num_actividades = driver.find_element("css selector", "#num_tours").text
    num_actividades = int(num_actividades)

    # Como hay que ir cargando el contenido vamos a calcular cuantas veces hay que dar al botón sabiendo que carga 20 actividades cada vez
    num_clicks=math.ceil(num_actividades/20)

    for i in range(num_clicks-1):
        driver.execute_script("window.scrollTo(0, 15000);")
        sleep(1)
        driver.find_element("css selector", "#more_tours > span").click()
        sleep(1)

    html_table_page = driver.page_source
    with open(f'../datos/actividades_{ciudades[j]}.html', 'w', encoding='utf-8') as file:
            file.write(html_table_page) 
    j+=1
    sleep(5)

driver.quit()

In [97]:
ciudades=["roma", "paris"]

for ciudad in ciudades:
    with open(f'../datos/actividades_{ciudad}.html', 'r', encoding='utf-8') as file:
        html_contenido = file.read()
    sopa = BeautifulSoup(html_contenido, "html.parser")

    # Sacamos las títulos
    lista_titulos = sopa.findAll("h2")
    titulos = [titulo.text for titulo in lista_titulos]

    # Sacamos las categorías
    lista_categorias = sopa.findAll("h3", {"class":"category desktop"})
    categorias = [categoria.text for categoria in lista_categorias]

    # Sacamos la puntuacion, las opiniones y si hay cancelación gratuita
    tour_list_desc = sopa.findAll("div", {"class":"tour_list_desc"})

    puntuacion=[]
    opiniones =[]
    cancelacion = []

    for tour in tour_list_desc:
        lista_puntuacion_aux=tour.select("div.tour_list_desc div.rating small") 
        lista_opiniones_aux=tour.select("div.tour_list_desc div.rating span")
        lista_cancelacion_aux=tour.select("div.tour_list_desc div.cancelacion-gratuita")

        # Sacamos la lista de las calificaciones
        if len(lista_puntuacion_aux)==0:
            puntuacion.append(np.nan)
        else:
            puntuacion.append(lista_puntuacion_aux[0].text.replace(" ", ""))

        # Sacamos la lista de las opiniones
        if len(lista_opiniones_aux)==0:
            opiniones.append(np.nan)
        else:
            opiniones.append(lista_opiniones_aux[0].text)

        # Sacamos la lista de si hay cancelacion gratuita
        if len(lista_cancelacion_aux)==0:
            cancelacion.append("No")
        else:
            cancelacion.append("Si")

    #Sacamos la duración de la actividad y el idioma
    duracion=[]
    idioma=[]

    tour_featured_aux = sopa.findAll("div", {"class":"price_list"})
    tour_featured = tour_featured_aux[::2]

    for tour in tour_featured:
        lista_duracion_aux=tour.select("div.price_list div.tduration")
        lista_idioma_aux=tour.select("div.price_list div.tlang")

        #Creamos la lista de duracion
        if len(lista_duracion_aux)==0:
            duracion.append(np.nan)
        else:
            duracion.append(lista_duracion_aux[0].text.replace("\n","").strip())

        #Creamos la lista de idiomas
        if len(lista_idioma_aux)==0:
            idioma.append(np.nan)
        else:
            idioma.append(lista_idioma_aux[0].text.replace("\n","").strip())

    #Sacamos el precio de la actividad
    precios=[]
    precios_aux = sopa.findAll("div", {"class":"price_container"})
    for precio in precios_aux:
        lista_precio = precio.findAll("span")
        #Es gratis
        if len(lista_precio)==2: 
            precios.append(0)
        #El precio no esta rebajado
        elif len(lista_precio)==3: 
            precios.append(float(lista_precio[0].text))
        #El precio si esta rebajado
        elif len(lista_precio)>3:
            precios.append(float(lista_precio[3].text))
        else:
            precios.append(np.nan)

    dic_df={"actividad":titulos, 
            "categoria":categorias, 
            "cancelacio_gratuita":cancelacion, 
            "puntuacion":puntuacion, 
            "opiniones":opiniones, 
            "duracion":duracion, 
            "idioma":idioma, 
            "precio":precios}

    df=pd.DataFrame(dic_df)
    df["opiniones"]=df["opiniones"].str.replace(" Opiniones)", "").str.replace("(","")
    
    df.to_csv(f"../datos/df_{ciudad}.csv")




In [6]:
df_roma=pd.read_csv("../datos/df_roma.csv", index_col=0)
df_roma.head()

,actividad,categoria,cancelacio_gratuita,puntuacion,opiniones,duracion,idioma,precio
0,Visita guiada Vaticano y la Capilla Sixtina s...,Museos y Monumentos,Si,"9,4/10",1215,3 horas,Español,113.35
1,Excursión a Pompeya desde Roma con acceso prio...,Excursiones de un día,Si,"9,8/10",254,13:30 horas aprox,Español,124.50
2,"Descubre el Coliseo, el monte Palatino y el Fo...",Tours a pie,Si,"9,7/10",255,2 horas 30 minutos,Español,50.65
3,Descubre Museos Vaticanos y la Capilla Sixtina...,Visitas Guiadas,Si,"9,8/10",443,3 horas aprox.,Español,83.00
4,"Tour a pie por el Coliseo, el Foro Romano y el...",Tours a pie,Si,"9,6/10",1352,3 horas aprox.,Español,58.90


In [7]:
df_paris=pd.read_csv("../datos/df_paris.csv", index_col=0)
df_paris.head()

,actividad,categoria,cancelacio_gratuita,puntuacion,opiniones,duracion,idioma,precio
0,Crucero panorámico por el río Sena,Paseos en barco,Si,"9,3/10",155,1 hora,Español,16.00
1,Autobús turístico de París Open Tour con parad...,Tours en Bus turístico,Si,"9,8/10",699,flexible,Español,43.55
2,Free tour París imprescindible,Tours a pie,Si,"9,7/10",233,2 horas 30 minutos,Español,0.00
3,Torre Eiffel sin colas y crucero por el río Sena,Entradas y Atracciones,Si,"9,7/10",839,3 horas (aprox.),NaN,74.15
4,Disneyland® Paris – Entradas de un día sin fecha,Entradas y Atracciones,No,"9,7/10",122,1 dia o 2 dias,NaN,110.30
